In [1]:
#Initilize GEE
import os
work_dir = os.path.expanduser('../data')


import ee
#geemap is just for ViZ
import geemap
import numpy

geemap.ee_initialize()

In [2]:
#need to check version nodeJ
# here running with, probably wanting v18.17.1
! node -v

v18.17.1


 use the following to fix version 

In [3]:
(gee) beber@fbk:~$ nvm use v18.17.1
Now using node v18.17.1 (npm v9.6.7)


then restart kernel jupyter

SyntaxError: invalid syntax (2215031909.py, line 1)

# coefficients and SAT re-calibration from: https://doi.org/10.3390/rs12091471
# cite:
Ermida, S.L., Soares, P., Mantas, V., Göttsche, F.M. and Trigo, I.F., 2020. Google earth engine open-source code for land surface temperature estimation from the landsat series. Remote Sensing, 12(9), p.1471.

In [3]:
import oeel

lib = geemap.requireJS('./landsat_smw_lst/modules/Landsat_LST.js') #modules/Landsat_LST.js')
print(lib.availability)

LandsatLST = lib

{'collection': 'function'}


In [4]:
# example of images acquired and processed
# select region of interest, date range, and landsat satellite
#
# zaragoza
#Box -1.209183,41.476175,15.906830,47.322069
# geometry =ee.Geometry.Polygon(
#         [[[-1.2, 41.9],
#           [-1.2, 41.3],
#           [-0.6, 41.3],
#           [-0.6, 41.9]]], None, False)
# Graz
#15.030670,46.733272,15.906830,47.322069
# geometry =ee.Geometry.Polygon(
#         [[[15.2, 46.9],
#           [15.2, 47.2],
#           [15.7, 47.2],
#           [15.7, 46.9]]], None, False)

# #Leuven
# geometry =ee.Geometry.Polygon(
#         [[[4.677773, 50.892098],
#           [4.677773, 50.860361],
#           [4.729271, 50.860361],
#           [4.729271, 50.892098]]], None, False)
#
#Ferrara
geometry =ee.Geometry.Polygon(
        [[[11.524777, 44.9044706],
          [11.524777, 44.76288],
          [11.6722516, 44.76288],
          [11.6722516, 44.9044706]]], None, False)

# #Trento
# geometry = ee.Geometry.Polygon(
#         [[[10.956465573588993, 46.178886859944434],
#           [10.956465573588993, 45.96738643146951],
#           [11.338240475932743, 45.96738643146951],
#           [11.338240475932743, 46.178886859944434]]], None, False)

satellite = 'L8';
date_start = '2013-01-01'; 
date_end = '2023-12-31';
use_ndvi = True

# get landsat collection with added variables: NDVI, FVC, TPW, EM, LST
LandsatColl = LandsatLST.collection(satellite, date_start, date_end, geometry, use_ndvi)

# select a feature to plot
exImage = LandsatColl.median()
# select the first feature
#exImage = LandsatColl.firtst()
#get last
#exImage=LandsatColl.limit(1, 'system:time_start', False).first()


cmap1 = ['blue', 'cyan', 'green', 'yellow', 'red']
cmap2 = ['F2F2F2','EFC2B3','ECB176','E9BD3A','E6E600','63C600','00A600']

Map = geemap.Map(Draw_export=True)

Map.centerObject(geometry)
#Map.addLayer(exImage.select('TPW'),{'min':0.0, 'max':60.0, 'palette':cmap1},'TCWV')
#Map.addLayer(exImage.select('TPWpos'),{'min':0.0, 'max':9.0, 'palette':cmap1},'TCWVpos')
#Map.addLayer(exImage.select('FVC'),{'min':0.0, 'max':1.0, 'palette':cmap2}, 'FVC')
#Map.addLayer(exImage.select('EM'),{'min':0.9, 'max':1.0, 'palette':cmap1}, 'Emissivity')
#Map.addLayer(exImage.select('B10'),{'min':290, 'max':320, 'palette':cmap1}, 'TIR BT')
Map.addLayer(exImage.select('LST'),{'min':290, 'max':320, 'palette':cmap1}, 'LST')
Map.addLayer(exImage.multiply(0.0000275).add(-0.2),{'bands': ['SR_B4',  'SR_B3',  'SR_B2'], 'min':0, 'max':0.3}, 'RGB')
Map.addLayer(geometry)

# # 

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# create Hot Spots maps by median over extended summer months for 2013-2023

In [17]:
# save to drive
# check task here: https://code.earthengine.google.com/tasks
# check your drive for images

for m in range(4, 10) :
    month_col = LandsatColl.filter(ee.Filter.calendarRange(m, m,'month'))
    m_median = month_col.select('LST').filter(ee.Filter.bounds(geometry)).median();
    filename = "Hot_spots_month_" + str(m)    # check download_ee_image VS ee_export_image
    geemap.ee_export_image_to_drive(m_median,folder='Zaragoza_HS',  description=filename, scale=30, region=geometry)

# create LST time series maps by median over scene available for each month for each year

In [115]:
#functions
# Function to get the images for a given year and month
def get_monthly_collection(LandsatColl, year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    #
    # Filter the Landsat 8 collection by date and apply median
    collection = LandsatColl.filterDate(start_date, end_date).median()
    #
    return collection

# need to manage months with no scenes
def comprobeBandsNumber(ele):
  new_list = ee.List([])
  count = ee.Image(ele).bandNames().size()
  # 16 is the number of bands in correct scene  
  comp = ee.Algorithms.If(count.eq(16), ele, 0)
  new_list = new_list.add(comp)
  return new_list
#

In [119]:
for m in range(1, 13) :
    # Create an empty list to store monthly collections
    monthly_collections = []
    #
    start_year = 2013
    end_year = 2023
    #
    # Loop through each year and month
    for year in range(start_year, end_year + 1):
        monthly_collection = get_monthly_collection(LandsatColl,year, m)
        monthly_collections.append(monthly_collection)
    #branching here
    # Merge all collections into one ImageCollection
    #this would work if you manage images
    sorted_monthly_collections = ee.ImageCollection.fromImages(monthly_collections) #.sort('system:time_start')
    # save collect.select('LST').ion LST into 1 image with bands as TS
    listOfImages = sorted_monthly_collections.toList(sorted_monthly_collections.size());

    # count zeros nad their pos
    newList = listOfImages.map(comprobeBandsNumber).flatten();
    x= numpy.array(newList.getInfo())
    zero_loc = numpy.where(x == 0)[0]

    #removing zeroes in new list
    newList = newList.removeAll([0]);
    #creating new collection
    sorted_monthly_collections_fixed = ee.ImageCollection(newList);
    
    LST_TS_13_23 = sorted_monthly_collections_fixed.select('LST').toBands()
    #renamen bands
    new_bands = ['LST_2013','LST_2014','LST_2015','LST_2016','LST_2017','LST_2018','LST_2019','LST_2020','LST_2021','LST_2022','LST_2023'];
    #
    #fix bands names based on scene availability
    new_bands = [x for i, x in enumerate(new_bands) if i not in zero_loc]
    LST_TS_13_23_fix = LST_TS_13_23.rename(new_bands)
    #
    filename = "LST_2013_2023_Time_series_month_" + str(m)    # check download_ee_image VS ee_export_image
    geemap.ee_export_image_to_drive(LST_TS_13_23_fix,folder='Ferrara_HS',  description=filename, scale=30, region=geometry)